In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("train.txt", sep=";", header=None, names=["text", "emotions"])

In [ ]:
df.head()

# data cleaning and preprocessing


In [ ]:
df.isnull().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.duplicated().sum()

In [ ]:
df["emotions"].unique()

In [ ]:
# convert emotions to numbers

unique_emotions = df["emotions"].unique()
emotion_numbers = {}
# Assign a unique number to each emotion
i = 0
for emotion in unique_emotions:
    emotion_numbers[emotion] = i
    i += 1

df["emotions"] = df["emotions"].map(emotion_numbers)

In [ ]:
emotion_numbers

In [ ]:
df.head()

In [ ]:
# converting all text to lower case
df["text"] = df["text"].str.lower()

In [ ]:
df.head()

In [ ]:
# removing punctuation
import string


def remove_punctuation(text):
    return text.translate(str.maketrans("", "", string.punctuation))


df["text"] = df["text"].apply(remove_punctuation)

In [ ]:
df.head()

In [ ]:
# removing numbers from text
def remove_numbers(text):
    new = ""
    for i in text:
        if not i.isdigit():
            new += i
    return new


df["text"] = df["text"].apply(remove_numbers)

In [ ]:
df.head()

In [ ]:
# removing urls/links from text
import re


def remove_urls(text):
    return re.sub(r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE)


df["text"] = df["text"].apply(remove_urls)

In [ ]:
df.head()

In [ ]:
# removing extra spaces from text
def remove_extra_spaces(text):
    return re.sub(r"\s+", " ", text).strip()


df["text"] = df["text"].apply(remove_extra_spaces)

In [ ]:
df.head()

In [ ]:
# removing emojis from text
def remove_emojis(text):
    new = ""
    for i in text:
        if i.isascii():
            new += i
    return new


df["text"] = df["text"].apply(remove_emojis)

In [ ]:
df.head()

In [ ]:
# removing stopwords using nltk library
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
# download stopwords
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("punkt_tab")

In [ ]:
eng_stopwords = set(stopwords.words("english"))

In [ ]:
len(eng_stopwords)

In [ ]:
type(eng_stopwords)

In [ ]:
df["text"].loc[1]

In [ ]:
# tokenzing and removing stopwords
def remove_stopwords(text):
    words = word_tokenize(text)
    cleaned_text = []

    for word in words:
        if word not in eng_stopwords:
            cleaned_text.append(word)

    # Join the cleaned words back into a single string
    return " ".join(cleaned_text)


df["text"] = df["text"].apply(remove_stopwords)

In [ ]:
df["text"].loc[1]

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
# separating features and labels
x = df["text"]
y = df["emotions"]

In [ ]:
# train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42
)

In [ ]:
X_train.shape, X_test.shape

## model training using bow


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
bow_vectorizer = CountVectorizer()

In [ ]:
X_train_bow = bow_vectorizer.fit_transform(X_train)
X_train_bow

In [ ]:
X_test_bow = bow_vectorizer.transform(X_test)
X_test_bow

In [ ]:
# logistic regression classifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
lr_model = LogisticRegression(max_iter=1000, n_jobs=-1)
lr_model.fit(X_train_bow, y_train)
y_pred = lr_model.predict(X_test_bow)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of the Logistic Regression model: {accuracy * 100:.2f}%")

In [ ]:
#saving the model
import joblib
joblib.dump(lr_model, "logistic_regression_model.pkl")
